In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from google.colab import files


In [4]:
uploaded = files.upload()

Saving TSSData_20201017.csv to TSSData_20201017.csv


In [2]:
traffic = pd.read_csv('TSSData_20201017.csv')

In [3]:
traffic.head()

,numberOfFeatures,timeStamp,ns3:id,ns2:detectorid,ns2:stationid,ns2:device_name,ns2:lane_direction,ns2:lanetype,ns2:lanenum,ns2:last_updated,ns2:mile_marker,ns2:occupancy,ns2:routename,ns2:speed,srsName,srsDimension,ns3:pos,ns2:volume,ns2:nodeid,ns2:sample_period
0,3354,2020-10-18T02:59:37.722Z,tss_detector_status.OpenTMS-Detector-580522384-0,OpenTMS-Detector-580522384-0,OpenTMS-Detector-580522384,LANE 1 SB OFF RAMP,S,OFF_RAMP,1,2020-10-18T02:59:00Z,176.0,2,I-95S,49,urn:x-ogc:def:crs:EPSG:4326,2,38.801299 -77.08368,2,0.0,60
1,3354,2020-10-18T02:59:37.722Z,tss_detector_status.HamptonRoads-TSS-L0642983-1_2,HamptonRoads-TSS-L0642983-1_2,HamptonRoads-TSS-L0642983-1,64WB after Military Hwy exit W Main Lane 2,W,Main,2,2020-07-19T23:23:00Z,0.0,7,I-64,57,urn:x-ogc:def:crs:EPSG:4326,2,36.77381 -76.36783,16,NaN,60
2,3354,2020-10-18T02:59:37.722Z,tss_detector_status.OpenTMS-Detector-580523190-7,OpenTMS-Detector-580523190-7,OpenTMS-Detector-580523190,S4,S,NORMAL,3,2020-10-18T02:58:00Z,40.6,3,VA-28N,64,urn:x-ogc:def:crs:EPSG:4326,2,38.98235 -77.42905,9,7.0,60
3,3354,2020-10-18T02:59:37.722Z,tss_detector_status.OpenTMS-Detector-399568-2,OpenTMS-Detector-399568-2,OpenTMS-Detector-399568,WB Middle Lane,W,NORMAL,2,2020-10-18T02:50:00Z,202.0,1,I-64W,71,urn:x-ogc:def:crs:EPSG:4326,2,37.51673 -77.2536,21,2.0,300
4,3354,2020-10-18T02:59:37.722Z,tss_detector_status.OpenTMS-Detector-580523190-8,OpenTMS-Detector-580523190-8,OpenTMS-Detector-580523190,S3,S,NORMAL,4,2020-10-18T02:58:00Z,40.6,1,VA-28N,65,urn:x-ogc:def:crs:EPSG:4326,2,38.98235 -77.42905,5,8.0,60


In [4]:
traffic.describe()

,numberOfFeatures,ns2:lanenum,ns2:mile_marker,ns2:occupancy,ns2:speed,srsDimension,ns2:volume,ns2:nodeid,ns2:sample_period
count,80496.0,80496.000000,75984.00000,80496.000000,80496.000000,80496.0,80496.000000,62232.000000,80496.000000
mean,3354.0,1.577818,71.71749,3.221067,55.344141,2.0,26.874602,1.998843,194.311270
std,0.0,1.305949,79.03163,4.580930,142.819777,0.0,50.861127,2.140566,128.839773
min,3354.0,0.000000,0.00000,0.000000,-8013.000000,2.0,0.000000,0.000000,60.000000
25%,3354.0,1.000000,0.10000,0.000000,57.000000,2.0,2.000000,0.000000,60.000000
50%,3354.0,1.000000,42.75000,2.000000,66.000000,2.0,10.000000,1.000000,300.000000
75%,3354.0,2.000000,161.10000,5.000000,71.000000,2.0,29.000000,3.000000,300.000000
max,3354.0,10.000000,324.00000,100.000000,10241.000000,2.0,506.000000,10.000000,900.000000


In [35]:
keep_cols = ['timeStamp', 'ns2:lanenum', 'ns2:occupancy', 'ns2:speed', 'ns2:volume', 'ns2:sample_period']

In [36]:
# traff = traffic[traffic['ns3:id'] == 'tss_detector_status.OpenTMS-Detector-580523190-7'][keep_cols]
traff = traffic[keep_cols]
traff['hour'] = pd.to_datetime(traff['timeStamp']).apply(lambda x: x.hour)
traff['min'] = pd.to_datetime(traff['timeStamp']).apply(lambda x: x.minute)
traff.drop('timeStamp', axis=1, inplace=True)
traff.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 

,ns2:lanenum,ns2:occupancy,ns2:speed,ns2:volume,ns2:sample_period,hour,min
0,1,2,49,2,60,2,59
1,2,7,57,16,60,2,59
2,3,3,64,9,60,2,59
3,2,1,71,21,300,2,59
4,4,1,65,5,60,2,59


In [37]:
train_dataset = traff.sample(frac=0.8, random_state=0)
test_dataset = traff.drop(train_dataset.index)

In [38]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('ns2:occupancy')
test_labels = test_features.pop('ns2:occupancy')

In [42]:
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(6,)),
        layers.Dense(30, activation="relu", name="layer1"),
        layers.Dense(30, activation="relu", name="layer2"),
        layers.Dense(1, name="layer3"),
    ]
)

model.build((6, 1))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 30)                210       
_________________________________________________________________
layer2 (Dense)               (None, 30)                930       
_________________________________________________________________
layer3 (Dense)               (None, 1)                 31        
Total params: 1,171
Trainable params: 1,171
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [44]:
history = model.fit(
    train_features, train_labels,
    epochs=10,
    validation_split = 0.2)

Epoch 1/10
1610/1610 [==============================] - 2s 1ms/step - loss: 4.2409 - val_loss: 2.8330
Epoch 2/10
1610/1610 [==============================] - 2s 1ms/step - loss: 3.0638 - val_loss: 2.8316
Epoch 3/10
1610/1610 [==============================] - 2s 1ms/step - loss: 2.8275 - val_loss: 2.8405
Epoch 4/10
1610/1610 [==============================] - 2s 1ms/step - loss: 2.8296 - val_loss: 2.8770
Epoch 5/10
1610/1610 [==============================] - 2s 1ms/step - loss: 2.8272 - val_loss: 2.8371
Epoch 6/10
1610/1610 [==============================] - 2s 1ms/step - loss: 2.8283 - val_loss: 2.8557
Epoch 7/10
1610/1610 [==============================] - 2s 1ms/step - loss: 2.8283 - val_loss: 2.8432
Epoch 8/10
1610/1610 [==============================] - 2s 1ms/step - loss: 2.8291 - val_loss: 2.8356
Epoch 9/10
1610/1610 [==============================] - 2s 1ms/step - loss: 2.8276 - val_loss: 2.8340
Epoch 10/10
1610/1610 [==============================] - 2s 1ms/step - loss: 2.830

,numberOfFeatures,ns2:lanenum,ns2:mile_marker,ns2:occupancy,ns2:speed,srsDimension,ns2:volume,ns2:nodeid,ns2:sample_period
count,80496.0,80496.000000,75984.00000,80496.000000,80496.000000,80496.0,80496.000000,62232.000000,80496.000000
mean,3354.0,1.577818,71.71749,3.221067,55.344141,2.0,26.874602,1.998843,194.311270
std,0.0,1.305949,79.03163,4.580930,142.819777,0.0,50.861127,2.140566,128.839773
min,3354.0,0.000000,0.00000,0.000000,-8013.000000,2.0,0.000000,0.000000,60.000000
25%,3354.0,1.000000,0.10000,0.000000,57.000000,2.0,2.000000,0.000000,60.000000
50%,3354.0,1.000000,42.75000,2.000000,66.000000,2.0,10.000000,1.000000,300.000000
75%,3354.0,2.000000,161.10000,5.000000,71.000000,2.0,29.000000,3.000000,300.000000
max,3354.0,10.000000,324.00000,100.000000,10241.000000,2.0,506.000000,10.000000,900.000000


In [12]:
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          64000     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
Total params: 164,106
Trainable params: 164,106
Non-trainable params: 0
_________________________________________________________________
